# Lab 1: Information Retrieval

__Students:__ benra741, enral465 

### Crawling



a) Get the webpage content by using functions in 
__[urllib module](https://docs.python.org/3/library/urllib.html#module-urllib)__.

Other libraries are also fine to achieve the crawling.

e.g. scrapy, beautifulsoup... 

In [ ]:
import urllib.request
x = urllib.request.urlopen('https://play.google.com/store/apps/category/GAME?hl=en').read().decode('utf-8')
#print(x)

b) Get app url by regular expression using functions from __[re module](https://docs.python.org/3/library/re.html?highlight=re#module-re)__.

A useful online regular expression check.
__[Check your regular expression first](https://regex101.com)__.

In [4]:
import re
appreg = r'href=\"(/store/apps/details.*?)\"'
appre = re.compile(appreg)
app_url_list = re.findall(appre,x)
app_url_list = list(set(app_url_list))
#print(len(app_url_list))
#print(app_url_list)

c) Access specific webpage to get description of each app and then store the description in files.

In [36]:
import re
descg = r'itemprop=\"description.*?\">.*?<div jsname=\".*?\">(\w.*?)<\/div>'
desc = re.compile(descg)


def gettingDescription(html):
    descg = r'itemprop=\"description.*?\">.*?<div jsname=\".*?\">(\w.*?)<\/div>'
    desc = re.compile(descg)
    desc_url = re.findall(desc,x)[0]
    remove_tags = re.compile('<.*?>')
    desc_url = re.sub(remove_tags, '', desc_url)
    desc_url = re.sub('https:\/\/.*?(\s|\')', '', desc_url)
    return desc_url.encode('utf8')

def getAppTitle(raw_html):
    titlePattern = "itemprop=\"name\"> <div class=\"id-app-title\" tabindex=\"0\">.*?</div>"
    title = re.findall(titlePattern, str(raw_html))
    title = cleanhtml(str(title))
    return(title)

def urlToDesc(url):
    descPattern = "itemprop=\"description.?\">.?<div jsname=\".?\">.?</div>"
    rawHTML_temp = urlopen(url).read().decode('utf8')
    desc_temp = re.findall(descPattern, str(rawHTML_temp))
    desc_temp = cleanhtml(str(desc_temp))
    title_temp = getAppTitle(rawHTML_temp)
    return title_temp, desc_temp


app_url_list
    
app_url_done = []

i = 0

while i < 5 :
    url = app_url_list[0]
    app_url_list = app_url_list[1:]
    app_url_done.append(url) # done list to avoid reviews

    x = urllib.request.urlopen('https://play.google.com' + url +"&hl=en").read().decode('utf8')
    nameg = r'<title id=\"main-title\">(.*?) - Android'
    name = re.compile(nameg)
    name2 = re.findall(name, x)[0]
    #print(name2)
    desc_url = gettingDescription(x)
    #print(desc_url)
    temp = re.findall(appre, x)
    temp = list(set(temp))
    app_url_list = [url for url in app_url_list + temp if url not in app_url_done]        

    #print(app_url_done)
    i = i + 1
    file_name = 'description_{}.txt'.format(name2)
    with open(file_name , 'w') as file:
        print(desc_url, file = file)
    



### Construct Inverted file index (Vector Model)



d) Preprocess text using NLP techniques from __[nltk module](http://www.nltk.org/py-modindex.html)__.

Using nltk.download(ID) to get the corpora if it is not downloaded before. __[nltk corpora](http://www.nltk.org/nltk_data/)__

In [1]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
#with open('students.tsv') as fin:

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\EjarWare\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [23]:
from nltk.tokenize import word_tokenize, sent_tokenize, RegexpTokenizer
from nltk.corpus import stopwords

stop = set(stopwords.words('english'))
stop.add('br')
tokenizer = RegexpTokenizer(r'\w+')

with open(file_name) as fin:
    for line in fin:
    #    tokens = [ word for sent in sent_tokenize(line) for word in word_tokenize(sent)]
        #tokens = tokenizer.tokenize(line) 
        tokens = [ token.lower() for token in tokens ]
        tokens = [ token for token in tokens if token not in stop]
        

print(tokens)  

['b', 'welcome', 'world', 'merge', 'farm', 'newest', 'farming', 'merge', 'game', 'gram', 'games', 'merge', 'farm', 'xe2', 'x80', 'x99t', 'average', 'farming', 'game', 'use', 'merging', 'mechanics', 'achieve', 'task', 'grow', 'farm', 'harvest', 'delicious', 'fruits', 'veggies', 'plant', 'tons', 'crops', 'across', 'farm', 'merge', 'crops', 'grow', 'bigger', 'plants', 'produce', 'fruit', 'veggies', 'take', 'massive', 'harvest', 'sell', 'grow', 'farm', 'even', 'waiting', 'let', 'xe2', 'x80', 'x99s', 'start', 'merging', 'farm', 'merge', 'farm', 'features', 'plant', 'amp', 'merge', 'crops', 'xe2', 'x80', 'xa2', 'plant', 'crops', 'varying', 'types', 'field', 'become', 'available', 'xe2', 'x80', 'xa2', 'merge', 'similar', 'crops', 'grow', 'upgrade', 'providing', 'larger', 'harvest', 'xe2', 'x80', 'xa2', 'bigger', 'plants', 'fruit', 'vegetables', 'produce', 'manage', 'farm', 'xe2', 'x80', 'xa2', 'farming', 'endless', 'means', 'xe2', 'x80', 'x99ll', 'need', 'manage', 'farm', 'daily', 'xe2', 'x80

...)Compute tdidf using functions from __[scikit-learn module](http://scikit-learn.org/stable/modules/classes.html)__.

eg. TfidfVectorizer is used for converting a collection of raw documents to a matrix of TF-IDF features.

In [47]:
from sklearn.feature_extraction.text import TfidfVectorizer
transvector = TfidfVectorizer()
corpus = [
     'This is the first document.',
     'This is the second second document.',
     'And this is the third one.',
     'Is this the first document?',]
tfidf1 = transvector.fit_transform(corpus)
print(tfidf1.toarray())
print(transvector.get_feature_names())


[[ 0.          0.46979139  0.58028582  0.38408524  0.          0.
   0.38408524  0.          0.38408524]
 [ 0.          0.27925389  0.          0.22830836  0.          0.87501037
   0.22830836  0.          0.22830836]
 [ 0.51184851  0.          0.          0.26710379  0.51184851  0.
   0.26710379  0.51184851  0.26710379]
 [ 0.          0.46979139  0.58028582  0.38408524  0.          0.
   0.38408524  0.          0.38408524]]
['and', 'document', 'first', 'is', 'one', 'second', 'the', 'third', 'this']


### Query Process

eg. "Dragon, Control, hero, running"

eg. "The hero controls the dragon to run."

